In [1]:
import geopandas as gpd

In [2]:
obec_okres_kraj = gpd.read_file(r'./source-files/obec_okres_kraj.geojson')
sportoviska = gpd.read_file(r'./source-files/1_klasifikovane.geojson')

In [3]:
print(obec_okres_kraj.crs)
print(sportoviska.crs)

EPSG:4326
EPSG:4326


In [4]:
# Priradi Obec (s tym okres aj kraj) tak, ze pre kazde sportovisko spocita obsah pretinania geometrie sportoviska a geometrie obce a priradi sportovisko tej obci, s ktorou ma najvacsi prienik

poloha_data = gpd.sjoin(sportoviska, obec_okres_kraj, how="inner", predicate="intersects")

poloha_data['intersection_area'] = poloha_data.apply(
    lambda row: row['geometry'].intersection(obec_okres_kraj.loc[row['index_right'], 'geometry']).area,
    axis=1
)

poloha_data.reset_index(inplace=True)
poloha_data = poloha_data.loc[poloha_data.groupby('id')['intersection_area'].idxmax()]
poloha_data = poloha_data.drop(['index_right', 'obec', 'okres', 'kraj', 'intersection_area'], axis = 1)
poloha_data

,index,id,abandoned,access,addr:postcode,addr:street,addr:streetnumber,amenity,building,contact:instagram,...,hokej,volejbal,plavanie,viacucelove,zdroj,status,geometry,obec_id,okres_id,kraj_id
6200,11558,node/10003968772,None,None,08001,Škultétyho,None,None,None,None,...,0,0,1,0,OpenStreetMap,Aktívne,POINT (21.24419 48.98785),1823,48.0,5.0
394,11559,node/10084161428,None,yes,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,POINT (18.74262 49.21629),2920,79.0,8.0
395,11560,node/10084161430,None,yes,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,POINT (18.73872 49.21501),2920,79.0,8.0
5225,11561,node/10159452050,None,yes,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,POINT (18.17113 48.55325),2279,65.0,4.0
3191,11562,node/10253818042,None,None,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,POINT (17.11560 48.15653),155,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12135,8611,way/998690024,None,None,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,"POLYGON ((19.81850 48.30302, 19.81889 48.30256...",2886,32.0,1.0
4776,8612,way/999288148,None,None,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,"POLYGON ((18.16784 48.90731, 18.16819 48.90732...",2303,67.0,6.0
4777,8613,way/999288152,None,None,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,"POLYGON ((18.16776 48.90543, 18.16800 48.90537...",2303,67.0,6.0
4778,8614,way/999288153,None,None,None,None,None,None,None,None,...,0,0,0,0,OpenStreetMap,Aktívne,"POLYGON ((18.16804 48.90597, 18.16787 48.90601...",2303,67.0,6.0


In [5]:
# Ak dojde k duplicite zaznamu o sportovisku -> nieco je zle -> vyhodi error

counts = poloha_data.groupby('id').size()
counts = counts[counts >= 2]

if not counts.empty:
    raise Exception('Pri priradovani obce doslo k zdvojeniu zaznamu o sportovisku')

In [6]:
# kontroluje ci vsetkym sportoviskam bola priradena obec, ak nie vyhodi chybu
filtered_df = poloha_data.dropna(subset=['obec_id', 'okres_id', 'kraj_id'])
removed_rows = poloha_data[~poloha_data.index.isin(filtered_df.index)]
if removed_rows.shape[0] > 0:
    raise Exception('Nejakym sportoviskam sa nepodarilo priradit obec')

In [7]:
poloha_data.to_file(r'./source-files/2_doplnena_obec.geojson', driver = 'GeoJSON')